# Homework # 8

## Primal versus Dual Problem

### 1.

Recall that $N$ is the size of the data set and $d$ is the dimensionality of the
input space. The original formulation of the hard-margin SVM problem (minimize
$\frac{1}{2}\mathbf{w}^\intercal\mathbf{w}$ subject to the inequality constraints), without going through the
Lagrangian dual problem, is

[a] a quadratic programming problem with $N$ variables

[b] a quadratic programming problem with $N + 1$ variables

[c] a quadratic programming problem with $d$ variables

[d] a quadratic programming problem with $d + 1$ variables

[e] not a quadratic programming problem

The problem has $N$ variable in $\mathbf{w}$ and one variable $b$.

Notice: The following problems deal with a real-life data set. In addition, the computational packages you use may employ different heuristics and require different tweaks.
This is a typical situation that a Machine Learning practitioner faces. There are
uncertainties, and the answers may or may not match our expectations. Although
this situation is not as ‘sanitized’ as other homework problems, it is important to go
through it as part of the learning experience.

## SVM with Soft Margins

In the rest of the problems of this homework set, we apply soft-margin SVM to
handwritten digits from the processed US Postal Service Zip Code data set. Download
the data (extracted features of intensity and symmetry) for training and testing:

http://www.amlbook.com/data/zip/features.train

http://www.amlbook.com/data/zip/features.test

(the format of each row is: digit intensity symmetry). We will train two types
of binary classifiers; one-versus-one (one digit is class $+1$ and another digit is class
$−1$, with the rest of the digits disregarded), and one-versus-all (one digit is class $+1$
and the rest of the digits are class $−1$).

The data set has thousands of points, and some quadratic programming packages
cannot handle this size. We recommend that you use the packages in libsvm:

http://www.csie.ntu.edu.tw/~cjlin/libsvm/

Implement SVM with soft margin on the above zip-code data set by solving
$$\min \frac{1}{2}\sum_{n=1}^N \sum_{m=1}^N \alpha_n \alpha_m y_n y_m K(\mathbf{x}_n, \mathbf{x}_m) - \sum_{n=1}^N \alpha_n$$
$$\begin{eqnarray}
\mbox{s.t. } & &\sum_{n=1}^N y_n \alpha_n = 0 \\
& &0 \leq \alpha_n \leq C, n = 1, \cdots, N
\end{eqnarray}$$
When evaluating $E_\mathrm{in}$ and $E_\mathrm{out}$ of the resulting classifier, use binary classification error.

Practical remarks:

(i) For the purpose of this homework, do not scale the data when you use libsvm or
other packages, otherwise you may inadvertently change the (effective) kernel and get
different results.

(ii) In some packages, you need to specify double precision.

(iii) In 10-fold cross validation, if the data size is not a multiple of 10, the sizes of the
10 subsets may be off by 1 data point.

(iv) Some packages have software parameters whose values affect the outcome. ML
practitioners have to deal with this kind of added uncertainty.

In [4]:
# download("http://work.caltech.edu/data/features.train", "features.train")
# download("http://work.caltech.edu/data/features.test", "features.test")

training_data = readdlm("features.train")
test_data = readdlm("features.test");

train_features = training_data[:,2:3]
train_labels = training_data[:,1]
test_features = test_data[:,2:3]
test_labels = test_data[:,1]

function n_vs_all(labels, n)
    nlabels = labels[:]
    nlabels[labels .== n] = 1
    nlabels[labels .!= n] = -1
    return nlabels
end

function filter_m_vs_n(features, labels, m, n)
    mids = ifelse(labels .== m, true, false)
    nids = ifelse(labels .== n, true, false)
    mlabels = 1*ones(sum(mids))
    nlabels = -1*ones(sum(nids))
    return [features[mids,:];features[nids,:]], [mlabels; nlabels]
end

filter_m_vs_n (generic function with 1 method)

## Polynomial Kernels

Consider the polynomial kernel $K(\mathbf{x}_n ,\mathbf{x}_m) = (1 + \mathbf{x}_n^\intercal \mathbf{x}_m)^Q$, 
where $Q$ is the degree of the polynomial.

### 2.
With $C = 0.01$ and $Q = 2$, which of the following classifiers has the highest
$E_\mathrm{in}$?

[a] 0 versus all

[b] 2 versus all

[c] 4 versus all

[d] 6 versus all

[e] 8 versus all

In [9]:
using ScikitLearn
@sk_import svm: SVC

function question2()
    for i in 0:2:8
        labels = n_vs_all(train_labels, i)
        clf = SVC(C=0.01, kernel="poly", degree=2, gamma=1.0, coef0=1.0)
        fit!(clf, train_features, labels)
        println(i, ": ", 1.0 - score(clf, train_features, labels))
    end
end

question2()

0: 0.10588396653408316
2: 0.10026059525442321
4: 0.08942531888629812
6: 0.09107118365107669
8: 0.074338225209162


### 3.
With $C = 0.01$ and $Q = 2$, which of the following classifiers has the lowest
$E_\mathrm{in}$?

[a] 1 versus all

[b] 3 versus all

[c] 5 versus all

[d] 7 versus all

[e] 9 versus all

In [10]:
using ScikitLearn
@sk_import svm: SVC

function question3()
    for i in 1:2:9
        labels = n_vs_all(train_labels, i)
        clf = SVC(C=0.01, kernel="poly", degree=2, gamma=1.0, coef0=1.0)
        fit!(clf, train_features, labels)
        println(i, ": ", 1.0 - score(clf, train_features, labels))
    end
end

question3()

1: 0.014401316691811772
3: 0.09024825126868741
5: 0.07625840076807022
7: 0.08846523110684401
9: 0.08832807570977919


### 4.
Comparing the two selected classifiers from Problems 2 and 3, which of the
following values is the closest to the difference between the number of support
vectors of these two classifiers?

In [11]:
using ScikitLearn
@sk_import svm: SVC

function question4()
    labels0 = n_vs_all(train_labels, 0)
    labels1 = n_vs_all(train_labels, 1)
    clf = SVC(C=0.01, kernel="poly", degree=2, gamma=1.0, coef0=1.0)
    fit!(clf, train_features, labels0)
    len0 = length(clf[:support_])
    fit!(clf, train_features, labels1)
    len1 = length(clf[:support_])
    return abs(len0 - len1)
end

question4()

1793

### 5.
Consider the 1 versus 5 classifier with $Q = 2$ and $C \in \{0.001,0.01,0.1,1\}$.
Which of the following statements is correct? Going up or down means strictly
so.

[a] The number of support vectors goes down when $C$ goes up.

[b] The number of support vectors goes up when $C$ goes up.

[c] $E_\mathrm{out}$ goes down when $$ goes up.

[d] Maximum $C$ achieves the lowest $E_\mathrm{in}$.

[e] None of the above

In [12]:
using ScikitLearn
@sk_import svm: SVC

function question5()
    features, labels = filter_m_vs_n(train_features, train_labels, 1, 5)
    t_features, t_labels = filter_m_vs_n(test_features, test_labels, 1, 5)
    for c in [0.001,0.01,0.1,1]
        clf = SVC(C=c, kernel="poly", degree=2, gamma=1.0, coef0=1.0)
        fit!(clf, features, labels)
        println("C:",c)
        println("# of sv:", length(clf[:support_]))
        println("E_in:", 1 - score(clf, features, labels))
        println("E_out:", 1 - score(clf, t_features, t_labels))
        println("---")
    end
end

question5()

C:0.001
# of sv:76
E_in:0.004484304932735439
E_out:0.01650943396226412
---
C:0.01
# of sv:34
E_in:0.004484304932735439
E_out:0.018867924528301883
---
C:0.1
# of sv:24
E_in:0.004484304932735439
E_out:0.018867924528301883
---
C:1.0
# of sv:24
E_in:0.0032030749519538215
E_out:0.018867924528301883
---


### 6.
In the 1 versus 5 classifier, comparing $Q = 2$ with $Q = 5$, which of the following
statements is correct?

[a] When $C = 0.0001$, $E_\mathrm{in}$ is higher at $Q = 5$.

[b] When $C = 0.001$, the number of support vectors is lower at $Q = 5$.

[c] When $C = 0.01$, $E_\mathrm{in}$ is higher at $Q = 5$.

[d] When $C = 1$, $E_\mathrm{out}$ is lower at $Q = 5$.

[e] None of the above

In [4]:
using ScikitLearn
@sk_import svm: SVC

function question6()
    features, labels = filter_m_vs_n(train_features, train_labels, 1, 5)
    t_features, t_labels = filter_m_vs_n(test_features, test_labels, 1, 5)
    for c in [0.0001, 0.001, 0.01, 1]
        for q in [2, 5]
                clf = SVC(C=c, kernel="poly", degree=q, gamma=1.0, coef0=1.0)
            fit!(clf, features, labels)
            println("C:",c)
            println("# of sv:", length(clf[:support_]))
            println("E_in:", 1 - score(clf, features, labels))
            println("E_out:", 1 - score(clf, t_features, t_labels))
            println("---")
        end
    end
end

question6()

C:0.0001
# of sv:236
E_in:0.008968609865470878
E_out:0.01650943396226412
---
C:0.0001
# of sv:26
E_in:0.004484304932735439
E_out:0.018867924528301883
---
C:0.001
# of sv:76
E_in:0.004484304932735439
E_out:0.01650943396226412
---
C:0.001
# of sv:25
E_in:0.004484304932735439
E_out:0.021226415094339646
---
C:0.01
# of sv:34
E_in:0.004484304932735439
E_out:0.018867924528301883
---
C:0.01
# of sv:23
E_in:0.0038436899423446302
E_out:0.021226415094339646
---
C:1.0
# of sv:24
E_in:0.0032030749519538215
E_out:0.018867924528301883
---
C:1.0
# of sv:21
E_in:0.0032030749519538215
E_out:0.021226415094339646
---


##  Cross Validation
In the next two problems, we will experiment with 10-fold cross validation for the
polynomial kernel. Because E cv is a random variable that depends on the random
partition of the data, we will try 100 runs with different partitions and base our
answer on how many runs lead to a particular choice.

### 7.
Consider the 1 versus 5 classifier with $Q = 2$. We use $E_\mathrm{cv}$ to select $C \in \{0.0001,0.001,0.01,0.1,1\}$.
If there is a tie in $E\mathrm{cv}$, select the smaller $C$. Within the 100 random runs, which of the following statements is correct?

[a] $C = 0.0001$ is selected most often.

[b] $C = 0.001$ is selected most often.

[c] $C = 0.01$ is selected most often.

[d] $C = 0.1$ is selected most often.

[e] $C = 1$ is selected most often.

In [3]:
using ScikitLearn
using ScikitLearn.CrossValidation: KFold
@sk_import svm: SVC

function question7()
    features, labels = filter_m_vs_n(train_features, train_labels, 1, 5)
    dist = Dict()
    for _ in 1:100
        kfold = KFold(length(labels), n_folds=10, shuffle=true)
        err_cnts = []
        for c in [0.0001, 0.001, 0.01, 0.1, 1]
            clf = SVC(C=c, kernel="poly", degree=2, gamma=1.0, coef0=1.0)
            err_cnt = 0
            for (train, valid) in kfold
                fit!(clf, features[train,:], labels[train])
                err_cnt += sum(predict(clf, features[valid,:]) .!= labels[valid])
            end
            push!(err_cnts, err_cnt)
        end
        _, winner = findmin(err_cnts)
        dist[winner] = get(dist, winner, 0) + 1
    end
    println(dist)
end

question7()

Dict{Any,Any}(4=>14,2=>51,3=>24,5=>11)


### 8.
Again, consider the 1 versus 5 classifier with $Q = 2$. For the winning selection
in the previous problem, the average value of $E_\mathrm{cv}$ over the 100 runs is closest to

[a] 0.001

[b] 0.003

[c] 0.005

[d] 0.007

[e] 0.009

In [24]:
using ScikitLearn
using ScikitLearn.CrossValidation: KFold
@sk_import svm: SVC

function question8()
    features, labels = filter_m_vs_n(train_features, train_labels, 1, 5)
    err_cnts = []
    clf = SVC(C=0.001, kernel="poly", degree=2, gamma=1.0, coef0=1.0)
    for _ in 1:100
        kfold = KFold(length(labels), n_folds=10, shuffle=true)
        err_cnt = 0
        for (train, valid) in kfold
            fit!(clf, features[train,:], labels[train])
            err_cnt += sum(predict(clf, features[valid,:]) .!= labels[valid])
        end
        push!(err_cnts, err_cnt)
    end
    println(mean(err_cnts) / length(labels))
end

question8()

0.004798206278026906


## RBF Kernel
Consider the radial basis function (RBF) kernel $K(\mathbf{x}_n,\mathbf{x}_m) = \exp(-\|\mathbf{x}_n − \mathbf{x}_m\|^2)$ in
the soft-margin SVM approach. Focus on the 1 versus 5 classifier.

### 9.
Which of the following values of $C$ results in the lowest E in ?

[a] $C = 0.01$

[b] $C = 1$

[c] $C = 100$

[d] $C = 10^4$

[e] $C = 10^6$ 

### 10.
Which of the following values of $C$ results in the lowest $E_\mathrm{out}$?
[a] $C = 0.01$

[b] $C = 1$

[c] $C = 100$

[d] $C = 10^4$

[e] $C = 10^6$ 

In [26]:
using ScikitLearn
@sk_import svm: SVC

function question9and10()
    features, labels = filter_m_vs_n(train_features, train_labels, 1, 5)
    t_features, t_labels = filter_m_vs_n(test_features, test_labels, 1, 5)
    for c in [0.01, 1, 100, 1e4, 1e6]
        clf = SVC(C=c, kernel="rbf", gamma=1.0)
        fit!(clf, features, labels)
        println("C:",c)
        println("# of sv:", length(clf[:support_]))
        println("E_in:", 1 - score(clf, features, labels))
        println("E_out:", 1 - score(clf, t_features, t_labels))
        println("---")
    end
end

question9and10()

C:0.01
# of sv:406
E_in:0.0038436899423446302
E_out:0.02358490566037741
---
C:1.0
# of sv:31
E_in:0.004484304932735439
E_out:0.021226415094339646
---
C:100.0
# of sv:22
E_in:0.0032030749519538215
E_out:0.018867924528301883
---
C:10000.0
# of sv:19
E_in:0.002562459961563124
E_out:0.02358490566037741
---
C:1.0e6
# of sv:17
E_in:0.0006406149903908087
E_out:0.02358490566037741
---


# Appendices A: LIBSVM.jl

The implementation is not good and very slow. Furthermore, the API is terrible. So, it is not recommended to use this package.

In [ ]:
#Pkg.clone("https://github.com/simonster/LIBSVM.jl")
#Pkg.build("LIBSVM")

In [ ]:
using LIBSVM

function question2()
    for i in 0:2:8
        labels = n_vs_all(train_labels, i)
        clf = svmtrain(labels, train_features', kernel_type=Int32(1), degree=Int32(2), gamma=1.0, coef0=1.0)
        predicted, _ = svmpredict(clf, train_features')
        println(i, ": ",sum(predicted .!= labels) / length(labels))
    end
end
question2()

# Appendices B: RCall.jl
We can use R's functionality from Julia.

In [1]:
using RCall
R"library(e1071)"

RCall.RObject{RCall.StrSxp}
[1] "e1071"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     


In [2]:
R"?svm"

RCall.RObject{RCall.StrSxp}


svm                   package:e1071                    R Documentation

_S_u_p_p_o_r_t _V_e_c_t_o_r _M_a_c_h_i_n_e_s

_D_e_s_c_r_i_p_t_i_o_n:

     ‘svm’ is used to train a support vector machine. It can be used
     to carry out general regression and classification (of nu and
     epsilon-type), as well as density-estimation. A formula interface
     is provided.

_U_s_a_g_e:

     ## S3 method for class 'formula'
     svm(formula, data = NULL, ..., subset, na.action =
     na.omit, scale = TRUE)
     ## Default S3 method:
     svm(x, y = NULL, scale = TRUE, type = NULL, kernel =
     "radial", degree = 3, gamma = if (is.vector(x)) 1 else 1 / ncol(x),
     coef0 = 0, cost = 1, nu = 0.5,
     class.weights = NULL, cachesize = 40, tolerance = 0.001, epsilon = 0.1,
     shrinking = TRUE, cross = 0, probability = FALSE, fitted = TRUE,
     ..., subset, na.action = na.omit)
     
_A_r_g_u_m_e_n_t_s:

 formula: a symbolic description of the mod

In [5]:
using RCall, DataFrames
R"library(e1071)"

function question2()
    for i in 0:2:8
        labels = n_vs_all(train_labels, i)
        df = DataFrame([train_features labels])
        R"df <- $df"
        R"df$x3 <- as.factor(df$x3)"
        R"clf <- svm(x3~x1+x2, data=df, type='C-classification', cost=0.01, kernel='polynomial', degree=2, gamma=1.0, coef0=1.0, scale=FALSE)"
        #R"plot(clf, df)"
        pred = rcopy(R"fitted(clf)")
        pred = map(x->parse(Int32,x), pred)
        println(i, ": ", sum(pred .!= labels)/length(labels))
    end
end

question2()

0: 0.10588396653408312
2: 0.10026059525442327
4: 0.08942531888629818
6: 0.09107118365107666
8: 0.07433822520916199


# Appendices C: MATLAB.jl
We can use MATLAB's functionality from Julia.

In [6]:
using MATLAB

function question2()
    for i in 0:2:8
        labels = n_vs_all(train_labels, i)
        
        mat"""
        x = $train_features;
        y = $labels;
        clf = fitcsvm(x, y, 'BoxConstraint', 0.01, 'KernelFunction', 'polynomial','PolynomialOrder', 2);
        $pred = predict(clf, x);   
        """
        println(i, ": ", sum(pred .!= labels)/length(labels))
    end
end

question2()

A MATLAB session is open successfully
0: 0.10588396653408312
2: 0.10026059525442327
4: 0.08942531888629818
6: 0.09107118365107666
8: 0.07433822520916199
